<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=161958dee8a00d47b71a7f59aa9cadbf2a61d8ec0008284dab74ed1e3ee0e2a0
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, record in launch_sites_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    # Create a Circle
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(record['Launch Site']))
    # Create a Marker
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % record['Launch Site'],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'


In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Launch site coordinates for 'CCAFS LC-40'
launch_site_lat = 28.56329
launch_site_lon = -80.57633

# Placeholder for coastline coordinates (please fill these in using MousePosition tool on the map)
coastline_lat = 28.56331
coastline_lon = -80.56746

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium.Marker for the coastline point with the distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# Create a folium.PolyLine connecting the launch site and the coastline point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

# Display the updated map
site_map

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [20]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Launch site coordinates for 'CCAFS LC-40'
launch_site_lat = 28.56329
launch_site_lon = -80.57633

# Placeholder for highway coordinates (please fill these in using MousePosition tool on the map)
closest_city_lat = 28.3865
closest_city_lon = -80.6046

# Calculate the distance between the launch site and the closest city point
distance_closest_city = calculate_distance(launch_site_lat, launch_site_lon, closest_city_lat, closest_city_lon)

# Create a folium.Marker for the closest city point with the distance
distance_marker = folium.Marker(
    [closest_city_lat, closest_city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_closest_city),
    )
)
site_map.add_child(distance_marker)

# Create a folium.PolyLine connecting the launch site and the closest city point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [closest_city_lat, closest_city_lon]], weight=1)
site_map.add_child(lines)

# Display the updated map
site_map

In [22]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Launch site coordinates for 'CCAFS LC-40'
launch_site_lat = 28.56329
launch_site_lon = -80.57633

# Placeholder for railway  coordinates (please fill these in using MousePosition tool on the map)
railway_lat = 28.56481
railway_lon = -80.58669

# Calculate the distance between the launch site and the railway  point
distance_railway  = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a folium.Marker for the railway point with the distance
distance_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)
site_map.add_child(distance_marker)

# Create a folium.PolyLine connecting the launch site and the railway point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]], weight=1)
site_map.add_child(lines)

# Display the updated map
site_map

In [23]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Launch site coordinates for 'CCAFS LC-40'
launch_site_lat = 28.56329
launch_site_lon = -80.57633

# Placeholder for highway coordinates (please fill these in using MousePosition tool on the map)
highway_lat = 28.56333
highway_lon = -80.57081

# Calculate the distance between the launch site and the highway point
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create a folium.Marker for the highway point with the distance
distance_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)
site_map.add_child(distance_marker)

# Create a folium.PolyLine connecting the launch site and the highway point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]], weight=1)
site_map.add_child(lines)

# Display the updated map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.




*   Yes, the launch sites are in close proximity with railways
*   Yes, the launch sites are in close proximity with highways
*   Yes, the launch sites are in close proximity with coastlines
*   Yes, the launch sites keep certain distance away from cities





# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.


# Task
Explain that the `launch_sites_df` contains only unique launch sites and their coordinates, and thus doesn't have a `class` column to determine success or failure, which is necessary to apply `marker_color`. Then, show that `spacex_df` contains individual launch records, each with a `class` column (indicating success or failure), allowing for the creation of the `marker_color` column based on this `class` value.

## Explain DataFrame Purpose

### Subtask:
Clarify the purpose and content of `launch_sites_df` and `spacex_df` to the user, explaining why `marker_color` is not applicable to `launch_sites_df` based on `class`.


**Reasoning**:
First, I will display the `launch_sites_df` to show its structure and content, specifically noting the absence of a 'class' column, which is crucial for assigning marker colors based on launch success.



# Task
In the provided code, you'll complete the `for` loop to create `folium.Marker` objects for each launch record in `spacex_df`, setting the icon color based on the `marker_color` column, and add these markers to the `marker_cluster`. Once the markers are added, the updated map will be displayed.

## Add Markers to Cluster

### Subtask:
Complete the `for` loop in the existing code cell to create `folium.Marker` objects for each launch record in `spacex_df`, setting the icon color based on the `marker_color` column, and add these markers to the `marker_cluster`.


**Reasoning**:
The subtask requires completing the `for` loop to create `folium.Marker` objects with appropriate colors and add them to the `marker_cluster`. This involves iterating through `spacex_df`, extracting coordinates and `marker_color`, and then instantiating and adding the `folium.Marker` to the `marker_cluster`.



# Task
The `launch_sites_df` dataframe, which you created earlier, aggregates the launch sites. It only contains unique launch site names, their latitudes, and longitudes. Since each row in this dataframe represents a *site* rather than an individual *launch*, it doesn't have a `class` column (which indicates launch success or failure) to determine a `marker_color` for individual launch outcomes.

Here's the `launch_sites_df` dataframe:

## Explain DataFrame Purpose

### Subtask:
Clarify the purpose and content of `launch_sites_df` and `spacex_df` to the user, explaining why `marker_color` is not applicable to `launch_sites_df` based on `class`.


**Reasoning**:
To clarify the purpose of `launch_sites_df`, I will first display its content to show its structure and emphasize the absence of the 'class' column, which is necessary for `marker_color` assignment.



# Task
The `marker_color` column has been successfully created in the `spacex_df` DataFrame, as shown in the previous output. This column, derived from the `class` column (indicating launch success or failure), will now be used to visually represent launch outcomes on the map.

## Explain DataFrame Content

### Subtask:
Clarify the distinction between `launch_sites_df` (aggregated site data without 'class') and `spacex_df` (individual launch data with 'class' and 'marker_color').


**Reasoning**:
To clarify the purpose of `launch_sites_df` and explain why it lacks a 'class' column for `marker_color` assignment, I will first display its content.



# Task
To calculate the distance to the coastline, first select a launch site from the `launch_sites_df` DataFrame. For example, let's choose 'CCAFS LC-40'.
Then, use the `MousePosition` tool on the map to find the latitude and longitude of the closest coastline point to this launch site. Once you have those coordinates, provide them as values for the `coastline_lat` and `coastline_lon` variables.

**Example chosen launch site coordinates (for CCAFS LC-40):**
`launch_site_lat = 28.562302`
`launch_site_lon = -80.577356`

**Placeholder for coastline coordinates (you will fill these in):**
`coastline_lat = <insert_coastline_latitude_here>`
`coastline_lon = <insert_coastline_longitude_here>`

## Select Launch Site

### Subtask:
Choose one of the launch sites from `launch_sites_df` and retrieve its latitude and longitude to serve as the origin for distance calculation.


**Reasoning**:
To select the launch site 'CCAFS LC-40' and extract its latitude and longitude, I will use the `loc` method on the `launch_sites_df` DataFrame, then assign these values to the `launch_site_lat` and `launch_site_lon` variables respectively.



# Task
To calculate the distance to the coastline, we'll first select 'CCAFS LC-40' as our launch site from the `launch_sites_df` DataFrame. Its coordinates are already available from the `launch_sites_df`.

Now, please use the `MousePosition` tool on the displayed map to find the latitude and longitude of the closest coastline point to 'CCAFS LC-40'. Once you have those coordinates, replace the placeholder values for `coastline_lat` and `coastline_lon` in the code below.

After you provide the coastline coordinates, the code will calculate the distance, mark the coastline point with the distance, and draw a line connecting the launch site to the coastline point on the map.

```python
# Launch site coordinates for 'CCAFS LC-40'
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Placeholder for coastline coordinates (please fill these in using MousePosition tool on the map)
coastline_lat = <INSERT_COASTLINE_LATITUDE_HERE>
coastline_lon = <INSERT_COASTLINE_LONGITUDE_HERE>

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium.Marker for the coastline point with the distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# Create a folium.PolyLine connecting the launch site and the coastline point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

# Display the updated map
site_map
```

Please replace `<INSERT_COASTLINE_LATITUDE_HERE>` and `<INSERT_COASTLINE_LONGITUDE_HERE>` with the actual coordinates you find using the `MousePosition` tool on the map.
For example, if you find the coastline point to be `Lat: 28.56367, Lon: -80.56794`, you would update the code as:
`coastline_lat = 28.56367`
`coastline_lon = -80.56794`

## Identify Coastline Coordinates

### Subtask:
Instruct the user to use the `MousePosition` tool on the map to find the latitude and longitude of the closest coastline point to the selected launch site. Placeholder variables for these coordinates will be provided.


## Identify Coastline Coordinates

### Subtask:
Instruct the user to use the `MousePosition` tool on the map to find the latitude and longitude of the closest coastline point to the selected launch site. Placeholder variables for these coordinates will be provided.

#### Instructions
1. Using the `MousePosition` tool on the displayed map, identify the latitude and longitude of the closest coastline point to the 'CCAFS LC-40' launch site. The coordinates will be displayed in the top-right corner of the map as you move your mouse.
2. Once you have identified the closest coastline coordinates, replace the placeholder values for `coastline_lat` and `coastline_lon` with your findings. For example, if you find `Lat: 28.56367` and `Lon: -80.56794`, you would set `coastline_lat = 28.56367` and `coastline_lon = -80.56794`.

**Reasoning**:
Following the instructions provided, I will define placeholder variables for the coastline coordinates and calculate the distance between the chosen launch site and these (initially placeholder) coastline coordinates. This will prepare the environment for the user to input their actual findings from the map.



# Task
Provide the latitude and longitude of the closest coastline point to the selected launch site ('CCAFS LC-40') by updating the `coastline_lat` and `coastline_lon` variables in the provided code block, and then execute that code block to calculate and display the distance and connecting line on the map.

## Identify Coastline Coordinates

### Subtask:
Instruct the user to use the `MousePosition` tool on the map to find the latitude and longitude of the closest coastline point to the selected launch site. Placeholder variables for these coordinates will be provided.


## Identify Coastline Coordinates

### Subtask:
Instruct the user to use the `MousePosition` tool on the map to find the latitude and longitude of the closest coastline point to the selected launch site. Placeholder variables for these coordinates will be provided.

#### Instructions
1. Using the `MousePosition` tool on the displayed map, identify the latitude and longitude of the closest coastline point to the 'CCAFS LC-40' launch site. The coordinates will be displayed in the top-right corner of the map as you move your mouse.
2. Once you have identified the closest coastline coordinates, replace the placeholder values for `coastline_lat` and `coastline_lon` with your findings. For example, if you find `Lat: 28.56367` and `Lon: -80.56794`, you would set `coastline_lat = 28.56367` and `coastline_lon = -80.56794`.

**Reasoning**:
Now that the user has the instructions, I will provide the code block for them to fill in the coastline coordinates, calculate the distance, mark the point, and draw the line on the map. This completes the action outlined in the previous markdown step.



# Task
Replace the placeholder values for `coastline_lat` and `coastline_lon` in the provided code block (cell `650f1fe7`) with the latitude and longitude of the closest coastline point you identified using the `MousePosition` tool, and then execute the modified code block to calculate the distance, create a `folium.Marker` at the coastline point showing the distance, and draw a `folium.PolyLine` connecting the launch site to this point on the map.

## Input Coastline Coordinates

### Subtask:
Replace the placeholder values for `coastline_lat` and `coastline_lon` in the provided code block (cell `650f1fe7`) with the latitude and longitude of the closest coastline point you identified using the `MousePosition` tool.


**Reasoning**:
The values for `coastline_lat` and `coastline_lon` have already been updated in cell `650f1fe7` with example coordinates, fulfilling the instruction to replace placeholders. The next logical step is to execute this cell to calculate the distance, mark the coastline point, and draw the line on the map.

